In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Load dataset
df = pd.read_csv("/content/drive/MyDrive/backpack_train.csv")

In [ ]:
# Handling missing values
for col in df.columns:
    if df[col].dtype == "object":  # Categorical
        df[col].fillna("Unknown", inplace=True)
    else:  # Numerical
        df[col].fillna(df[col].median(), inplace=True)

<ipython-input-8-11fd1fd36ba6>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
<ipython-input-8-11fd1fd36ba6>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [ ]:
# Encode categorical variables
categorical_cols = ["Brand", "Material", "Size", "Laptop Compartment", "Waterproof", "Style", "Color"]
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Normalize numerical features
numerical_cols = ["Compartments", "Weight Capacity (kg)"]
scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
# Define dataset class
class TabularDataset(Dataset):
    def __init__(self, data):
        self.X = data.drop(columns=["id", "Price"]).values.astype(np.float32)
        self.y = data["Price"].values.astype(np.float32).reshape(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

# Train-test split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = TabularDataset(train_data)
test_dataset = TabularDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define TabTransformer model
class TabTransformer(nn.Module):
    def __init__(self, input_dim, embed_dim=32, num_heads=4, num_layers=2, hidden_dim=128):
        super(TabTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)

In [ ]:
# Model initialization
input_dim = len(df.columns) - 2  # Exclude 'id' and 'Price'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TabTransformer(input_dim).to(device)

# Training setup
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
train_model(model, train_loader)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 1643.4105
Epoch 2, Loss: 1528.7009
Epoch 3, Loss: 1528.6039
Epoch 4, Loss: 1528.2146
Epoch 5, Loss: 1528.4712
Epoch 6, Loss: 1528.5125
Epoch 7, Loss: 1528.7974
Epoch 8, Loss: 1528.3925
Epoch 9, Loss: 1528.5944
Epoch 10, Loss: 1528.6108
Epoch 11, Loss: 1528.2370
Epoch 12, Loss: 1528.3675
Epoch 13, Loss: 1528.5373
Epoch 14, Loss: 1529.1655
Epoch 15, Loss: 1528.4359
Epoch 16, Loss: 1528.1358
Epoch 17, Loss: 1528.2438
Epoch 18, Loss: 1528.4466
Epoch 19, Loss: 1528.4967
Epoch 20, Loss: 1528.4299
Epoch 21, Loss: 1528.6310
Epoch 22, Loss: 1528.4564
Epoch 23, Loss: 1528.2837
Epoch 24, Loss: 1528.4010
Epoch 25, Loss: 1528.4011
Epoch 26, Loss: 1528.2405
Epoch 27, Loss: 1528.6371
Epoch 28, Loss: 1528.4445
Epoch 29, Loss: 1528.6654
Epoch 30, Loss: 1528.5267
Epoch 31, Loss: 1528.2976
Epoch 32, Loss: 1528.3648
Epoch 33, Loss: 1528.7979
Epoch 34, Loss: 1528.6002
Epoch 35, Loss: 1528.2299
Epoch 36, Loss: 1528.2813
Epoch 37, Loss: 1528.2596
Epoch 38, Loss: 1528.6065
Epoch 39, Loss: 1528.

In [ ]:
# Performance tracking
# def evaluate_model(model, test_loader):
#     model.eval()
#     predictions, actuals = [], []
#     with torch.no_grad():
#         for X_batch, y_batch in test_loader:
#             X_batch = X_batch.to(device)
#             y_pred = model(X_batch).cpu().numpy()
#             predictions.extend(y_pred)
#             actuals.extend(y_batch.numpy())

#     predictions = np.array(predictions).flatten()
#     actuals = np.array(actuals).flatten()
#     mae = np.mean(np.abs(predictions - actuals))
#     rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
#     r2 = 1 - (np.sum((actuals - predictions) ** 2) / np.sum((actuals - np.mean(actuals)) ** 2))
#     print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).cpu().numpy().flatten()  # Flatten to match shapes
            y_batch = y_batch.cpu().numpy().flatten()

            predictions.extend(y_pred)
            actuals.extend(y_batch)

    # Convert to NumPy arrays
    predictions = np.array(predictions)
    actuals = np.array(actuals)

    # Ensure both arrays have the same length
    min_len = min(len(predictions), len(actuals))
    predictions = predictions[:min_len]
    actuals = actuals[:min_len]

    # Compute Metrics
    mae = np.mean(np.abs(predictions - actuals))
    rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
    r2 = 1 - (np.sum((actuals - predictions) ** 2) / np.sum((actuals - np.mean(actuals)) ** 2))

    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

evaluate_model(model, test_loader)


MAE: 32.9696, RMSE: 38.2751, R²: -0.0003
